In [0]:
# fixed issue reading blah blah...
# fixed 2 of 5 issues

In [0]:
dbutils.widgets.removeAll()

In [0]:
# Convert CSV files into PArquet format
# Delta Lake: Move data from Bronze to Silver
# Row based CSV foramt to Column based Parquet

dbutils.widgets.text("ratingsPath", "abfss://bronze@gksdatalake.dfs.core.windows.net/ratings/")
dbutils.widgets.text("ratingsTargetPath", "abfss://silver@gksdatalake.dfs.core.windows.net/ratings/")

In [0]:
RATINGS_PATH = dbutils.widgets.get("ratingsPath") 
print (RATINGS_PATH)

In [0]:
# how to create schema programatically instead of using inferSchema
from pyspark.sql.types import StructType, LongType, StringType, IntegerType, DoubleType

In [0]:
# True is nullable, False is non nullable
# give your own column name, datatypes
ratingSchema = StructType()\
                .add("userId", IntegerType(), True)\
                .add("movieId", IntegerType(), True)\
                .add("rating", DoubleType(), True)\
                .add("timestamp", LongType(), True)

In [0]:
# movieDf with schema we have, to avoid additional job creation, scan data overload
# since we provide schema, header = true, is to skip the first line in the csv
ratingDf = (spark
            .read
            .option("header", True)
            .schema(ratingSchema) # now , we don't run a job for schema
            .csv(RATINGS_PATH)
            )
ratingDf.printSchema()
ratingDf.show(5)
# make a note of number of jobs its create, compare with previous shell

In [0]:
# ratings data, convert to parquet as is
# silver container
# movies directory

# ratings to silver container
 
RATINGS_TARGET_PATH = dbutils.widgets.get("ratingsTargetPath")
# overwrite - delete old data
# add a filter, if ratings <= 0.5, do not move to silver

(ratingDf
    .filter ("rating > 0.5")
    .write
    .mode("overwrite")
    .parquet(RATINGS_TARGET_PATH))

In [0]:
dbutils.jobs.taskValues.set(key = "ratings_silver_path", value = RATINGS_TARGET_PATH)
